<a href="https://colab.research.google.com/github/Happday-bot/googlegenai/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np

In [ ]:
words ={}

In [ ]:
input_text = '''Title: Nelson Mandela: A Legacy of Freedom and Equality

Nelson Rolihlahla Mandela, born on July 18, 1918, in the small village of Mvezo,
 in South Africa, is one of the most iconic figures of the 20th century.
 His journey from a young boy in rural Africa to becoming the first black president of South Africa and a global symbol
 of resistance against racial oppression is a story of incredible resilience, sacrifice, and an unwavering commitment to justice.
 Mandela’s life is a testament to the power of hope and perseverance in the face of unimaginable adversity.'''

In [ ]:
sentences = re.split('[.,]', input_text)

In [ ]:
sentences

['Title: Nelson Mandela: A Legacy of Freedom and Equality\n\nNelson Rolihlahla Mandela',
 ' born on July 18',
 ' 1918',
 ' in the small village of Mvezo',
 '\n in South Africa',
 ' is one of the most iconic figures of the 20th century',
 ' \n His journey from a young boy in rural Africa to becoming the first black president of South Africa and a global symbol \n of resistance against racial oppression is a story of incredible resilience',
 ' sacrifice',
 ' and an unwavering commitment to justice',
 ' \n Mandela’s life is a testament to the power of hope and perseverance in the face of unimaginable adversity',
 '']

In [ ]:
for i in range(len(sentences)):
  sentences[i] = sentences[i].replace('\n', ' ')
  sentences[i] = sentences[i].replace("'", " ' ")
  sentences[i] = sentences[i].replace("’", " ’ ")
  sentences[i] = sentences[i].lower()
  if sentences[i] == '' or sentences[i] == ' ':
    sentences.pop(i)



In [ ]:
sentences

['title: nelson mandela: a legacy of freedom and equality  nelson rolihlahla mandela',
 ' born on july 18',
 ' 1918',
 ' in the small village of mvezo',
 '  in south africa',
 ' is one of the most iconic figures of the 20th century',
 '   his journey from a young boy in rural africa to becoming the first black president of south africa and a global symbol   of resistance against racial oppression is a story of incredible resilience',
 ' sacrifice',
 ' and an unwavering commitment to justice',
 '   mandela ’ s life is a testament to the power of hope and perseverance in the face of unimaginable adversity']

In [ ]:
sentences = ' '.join(sentences)

In [ ]:
sentences

'title: nelson mandela: a legacy of freedom and equality  nelson rolihlahla mandela  born on july 18  1918  in the small village of mvezo   in south africa  is one of the most iconic figures of the 20th century    his journey from a young boy in rural africa to becoming the first black president of south africa and a global symbol   of resistance against racial oppression is a story of incredible resilience  sacrifice  and an unwavering commitment to justice    mandela ’ s life is a testament to the power of hope and perseverance in the face of unimaginable adversity'

In [ ]:
sentences = sentences.split()

In [ ]:
sentences.append('end_of_token')

In [ ]:
vec = np.random.uniform(-1, 1, 2106)

In [ ]:
vec

array([ 0.42411253, -0.29971712, -0.12634286, ...,  0.09172815,
        0.03012982, -0.79639791])

In [ ]:
sum(vec)

-30.221399909811414

In [ ]:
vec /= np.sum(vec)

In [ ]:
vec

array([-0.01403352,  0.00991738,  0.00418058, ..., -0.00303521,
       -0.00099697,  0.02635212])

In [ ]:
for i in range(len(sentences)):
  if sentences[i] not in words:
    words[sentences[i]] = np.random.uniform(-1, 1, 2106)
    words[sentences[i]] /= np.sum(words[sentences[i]])

# Skip gram type of data

In [ ]:
data = []
for i in range(len(sentences)):
  context_window = 6;
  consider_output = []
  left_side_words = [x for x in sentences[max(0,i-context_window//2):i]]
  right_side_words = [x for x in sentences[i+1:min(len(sentences),i+1+context_window//2)]]

  if len(left_side_words) > 0:
    consider_output = words[left_side_words[0]]

  for j in range(1,len(left_side_words)):
    consider_output = np.add(words[left_side_words[j]], consider_output)

  if len(left_side_words) > 0 and len(right_side_words)>0:
    consider_output = np.add(words[right_side_words[0]],consider_output)
  elif len(right_side_words)>0:
    consider_output = words[right_side_words[0]]

  for j in range(1,len(right_side_words)):
    consider_output = np.add(words[right_side_words[j]], consider_output)
  data.append({tuple(words[sentences[i]]):consider_output})


# **Writing the neural network**
# code from the Updated classifynumbers.ipynb file from this account

In [ ]:
import tensorflow as tf
import numpy as np
import random

In [ ]:
n_layers = 2
epoch = 6
w_input = [len(consider_output),4000]
w_output = [4000,len(consider_output)]
b_input = [4000,len(consider_output)]
b_output = [4000,len(consider_output)]

W=[]
B=[]

In [ ]:
def create_network(n_layers,W,B,w_input,w_output,b_input,b_output):
  for i in range(n_layers):
    # weight initialisation

    # Define matrix size
    n_in = w_input[i]
    n_out = w_output[i]

    # Calculate the range for Xavier initialization
    limit = np.sqrt(6) / np.sqrt(n_in + n_out)

    W.append(tf.random.uniform((n_in, n_out), minval=-limit, maxval=limit))

    # bias initialisation

    # Define matrix size
    n_in = b_input[i]
    n_out = b_output[i]

    # Calculate the range for Xavier initialization
    limit = np.sqrt(6) / np.sqrt(n_in + n_out)

    B.append(tf.random.uniform((1, n_out), minval=-limit, maxval=limit))

In [ ]:
def forward_pass(input,W,B,N):
  return tf.nn.relu(np.add(np.dot(input,W[N]),B[N]))

In [ ]:
def backward_pass(actual_output,pred_output,output,W,B,N,learning_rate):
  if N==0:
    return 0
  error = np.subtract(actual_output,pred_output)
  gradient_decent = pred_output * (1 - pred_output)
  delta = error * gradient_decent * learning_rate
  B[N] = np.add(B[N],delta)
  W[N] = np.add(W[N],np.dot(tf.transpose(output[N-1]),delta))
  prev_actual_output = [[0 for x in range(len(output[N-1][0]))]]
  for i in range(len(delta[0])):
    temp = []
    for j in range(len(output[N-1][0])):
      temp.append(output[N-1][0][j] * (1-output[N-1][0][j]) * delta[0][i])
    prev_actual_output = np.add(prev_actual_output,temp)
  backward_pass(prev_actual_output,output[N-1],output,W,B,N-1,learning_rate)

In [ ]:
def send_input(input,W,B,n_layers,output):
  for i in range(n_layers):
    if i==0:
      output.append(forward_pass(input,W,B,i))
    else:
      output.append(forward_pass(output[i-1],W,B,i))

In [ ]:
def nn_forward(actual_input,W,B,n_layers):
  output = []
  send_input(actual_input,W,B,n_layers,output)
  return output
def nn_backward(W,B,n_layers,normalised_actual_output,normalised_output,learning_rate):
  learning_rate = 1
  backward_pass(normalised_actual_output,normalised_output[n_layers-1],normalised_output,W,B,n_layers-1,learning_rate)

In [ ]:
def batch_avg_normaliser(actual,pred):
  temp_actual = actual[0]
  temp_pred = pred[0]
  for i in range(1, len(actual)):
    temp_actual = np.add(temp_actual,actual[i])

  for i in range(1, len(pred)):
    for j in range(len(pred[i])):
      temp_pred[j] = np.add(temp_pred[j],pred[i][j])
  actual = np.divide(temp_actual,len(actual))
  pred = []
  for i in range(len(temp_pred)):
    pred.append(np.divide(temp_pred[i],len(actual)))
  return actual,pred

In [ ]:
def batch_spliter(data):
  print("processing...")
  batch_size = 3
  np.random.shuffle(data)
  div = len(data)//batch_size
  for i in range(div):
    batch = data[i*batch_size:(i+1)*batch_size]
    batch_output = []
    batch_actual_output = []
    for j in batch:
      for k in j:
        actual_input = np.array(k)
        batch_actual_output.append(np.array(j[k]))
        batch_output.append(nn_forward(actual_input,W,B,n_layers))
    batch_actual_output , batch_output = batch_avg_normaliser(batch_actual_output,batch_output)

    learning_rate = 1

    nn_backward(W,B,n_layers,batch_actual_output,batch_output,learning_rate)

In [ ]:
def predict(input,W,B,n_layers):
  pred_output = []
  for i in range(n_layers):
    if i==0:
      pred_output.append(forward_pass(input,W,B,i))
    else:
      pred_output.append(forward_pass(pred_output[i-1],W,B,i))
  return pred_output[n_layers-1]

In [ ]:
create_network(n_layers,W,B,w_input,w_output,b_input,b_output)

In [ ]:
for i in range(epoch):
  print("EPOCH :",i+1)
  batch_spliter(data)
  print("processed epoch",i+1)

EPOCH : 1
processing...
processed epoch 1
EPOCH : 2
processing...
processed epoch 2
EPOCH : 3
processing...
processed epoch 3
EPOCH : 4
processing...
processed epoch 4
EPOCH : 5
processing...
processed epoch 5
EPOCH : 6
processing...
processed epoch 6


In [ ]:
import csv
tf.experimental.numpy.experimental_enable_numpy_behavior()

def save_weights_and_bias_to_csv(weights, bias, file_prefix):
    # Save the weight matrix
    weights_file = file_prefix + "_weights.csv"
    np.savetxt(weights_file, weights, delimiter=",", fmt="%.6f")
    print(f"Weights saved to {weights_file}")

    # Save the bias vector
    bias_file = file_prefix + "_bias.csv"
    np.savetxt(bias_file, bias.reshape(1, -1), delimiter=",", fmt="%.6f")
    print(f"Bias saved to {bias_file}")



In [ ]:
import pickle
with open('words', 'wb') as file:
    pickle.dump(words, file)

In [ ]:
with open('words', 'rb') as file:
    loaded_dict = pickle.load(file)

print(loaded_dict)

{'in': array([ 0.03023014, -0.0328696 ,  0.01088362, ..., -0.04019416,
       -0.01357347, -0.04534678]), '1990': array([ 0.10987232,  0.13309931, -0.05106468, ..., -0.04130014,
        0.05211141,  0.10209018]), 'after': array([-0.07239091,  0.06625953, -0.01189149, ...,  0.09041256,
       -0.05789499,  0.06864289]), '27': array([ 0.00361581,  0.00972301,  0.01166485, ...,  0.0291112 ,
        0.03063503, -0.04127975]), 'years': array([ 0.00191419, -0.00020488, -0.00078209, ..., -0.00382933,
       -0.00152884,  0.01125497]), 'of': array([-0.02253813,  0.0202739 ,  0.04508838, ...,  0.08365545,
        0.08411484,  0.06375308]), 'imprisonment': array([-0.03631459, -0.06929404, -0.01395782, ...,  0.07010084,
        0.03250318,  0.06169486]), 'nelson': array([ 0.0121048 , -0.0330327 , -0.00591223, ...,  0.01850266,
       -0.00198471,  0.01300166]), 'mandela': array([ 0.03092289, -0.02894009,  0.01070432, ..., -0.00735793,
        0.03318538,  0.00675994]), 'was': array([ 0.00483275, 

In [ ]:
save_weights_and_bias_to_csv(W[0],B[0], "layer1")
save_weights_and_bias_to_csv(W[1],B[1], "layer2")

Weights saved to layer1_weights.csv
Bias saved to layer1_bias.csv
Weights saved to layer2_weights.csv
Bias saved to layer2_bias.csv


In [ ]:
def load_weights_and_bias_from_csv(weights_file, bias_file):
    # Load the weight matrix
    weights = np.loadtxt(weights_file, delimiter=",")
    print(f"Weights loaded from {weights_file}")

    # Load the bias vector
    bias = np.loadtxt(bias_file, delimiter=",").flatten()  # Flatten to ensure 1D array
    print(f"Bias loaded from {bias_file}")

    return weights, bias

In [ ]:
weights_file = "layer1_weights.csv"
bias_file = "layer1_bias.csv"

W[0], B[0] = load_weights_and_bias_from_csv(weights_file, bias_file)

weights_file = "layer1_weights.csv"
bias_file = "layer1_bias.csv"

W[0], B[0] = load_weights_and_bias_from_csv(weights_file, bias_file)

Weights loaded from layer1_weights.csv
Bias loaded from layer1_bias.csv
Weights:
 [[ 0.025751  0.004074 -0.020283 ... -0.001982 -0.006074  0.008556]
 [-0.029745  0.028398  0.026305 ... -0.001691 -0.001969 -0.018734]
 [-0.024593 -0.026645  0.024859 ... -0.024315 -0.031186 -0.011017]
 ...
 [ 0.004256 -0.02576   0.01375  ...  0.025949 -0.023383 -0.009224]
 [ 0.028341  0.011786  0.029086 ... -0.011705 -0.00673   0.016518]
 [-0.005886 -0.025363 -0.011046 ...  0.027958  0.006634  0.027689]]
Bias:
 [-0.022956  0.020235  0.013875 ... -0.009459 -0.026704 -0.019848]
